# Notebook

Einfach alle Zellen ausführen, dann entsteht unten ein Widget indem ihr Plots von zwei Spalten gegeneinander anschauen könnt. 
Die Zelle unter "Spark starten" startet eine Spark Session, die Zelle unter "Spark stoppen" beendet diese. 
Ihr braucht die Spark Session für den Abschnitt "Laden und in Pandas verwandeln", das Widget an sich funktioniert ohne Spark. 

Es kann sein dass ihr nachdem ihr die Zelle unter "Pandas installieren" ausgeführt habt das Notebook neu starten müsst. (Details sind dort beschrieben). 

# Spark starten

In [1]:
import pyspark
import os
myAppName = "plot-widget" #Nur kleine Buchstaben hier, at any cost
spark = pyspark.sql.SparkSession\
       .builder\
       .appName(myAppName)\
       .config("spark.kubernetes.executor.podNamePrefix", os.environ["POD_NAME"]+'-'+myAppName)\
       .config("spark.executor.cores", "2")\
       .config("spark.executor.memory", "2g")\
       .config("spark.driver.maxResultSize", "2g")\
       .config("spark.dynamicAllocation.maxExecutors", "8")\
       .config("spark.jars.packages", "")\
       .getOrCreate()


# Packages installieren
Um die Plots auszuführen wird das python package pandas und matplotlib benötigt. Es muss **einmal** für euren User installiert werden. Also beim ersten ausführen des Notebooks die folgende Zelle ausführen. Danach am besten auskommentieren ('#' davor) oder einfach löschen, ihr braucht sie nicht mehr. Es kann sein dass ihr danach das Notbeook neu starten müsst, links oben "kernel --> Restart kernel". Die Fehlermeldung "ERROR: jupyterlab-git 0.21.0rc0 has requirement nbdime~=2.0, but you'll have nbdime 1.1.0 which is incompatible." is kein Problem so lange am Ende "Successfully i

In [ ]:
%pip install pandas matplotlib

# Imports

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Laden und in Pandas verwandeln

In [5]:
# Der Maschinentyp ist RZM oder VE, nummer 300 - 303, einfach hinten in filepath ändern
Machine = spark.read.parquet("/reif/01-data-corpus/303/RZM/filled.parquet")

In [6]:
# Der Filter hier ist auf den 1.April 2020 gesetzt. Ihr könnt den anpassen, sollte klar sein wie. 
#Aber Achtung, wenn ihr den Zeitraum zu lang macht kanns sein dass die Konvertierung in Pandas nicht mehr funtioniert und der Plot wird unübersichtlich
Machine_df = Machine.filter(F.col("timestamp") >= "2019-07-01 19:19:00").filter(F.col("timestamp") <= "2019-07-02 15:59:59").toPandas()

### Die folgende Zelle is optional, sie wirft die Spalten raus die nur einen Wert schreiben

In [ ]:
for col in Machine_df.columns:
    if len(Machine_df[col].unique()) < 2:
        Machine_df = Machine_df.drop(col, axis = 1)

# Plot eines Tages
Plottet die Werte aus zwei Spalten in den gleichen Plot. Wenn ihr "Same Axes" wählt wird für die Werte eine gemeinsame Skala (links) angelegt, bei "Different Axes" wird für den zweiten Plot rechts eine eigene Skala angelegt.

In [7]:
@interact 
def hallowelt_1(axis_types = ["Different Axes", "Same Axes"], column1 = Machine_df.drop("timestamp", axis = 1).columns, column2 = Machine_df.drop("timestamp", axis = 1).columns): 
    fig, ax1 = plt.subplots(figsize=(20,10))
    Machine_df.plot(x="timestamp", y= column1, ax = ax1)
    if axis_types == "Same Axes":
        Machine_df.plot(x="timestamp", y=column2, ax = ax1, color = "orange")
    else:
        ax1.legend(loc = "upper left")
        ax2 = ax1.twinx()
        Machine_df.plot(x="timestamp", y=column2, ax = ax2, color = "orange")
        ax2.legend(loc = "upper right")

interactive(children=(Dropdown(description='axis_types', options=('Different Axes', 'Same Axes'), value='Diffe…

In [9]:
@interact 
def hallowelt_2(column1 = Machine_df.drop("timestamp", axis = 1).columns): 
    fig, ax1 = plt.subplots(figsize=(20,10))
    Machine_df.plot(x="timestamp", y= column1, ax = ax1)

interactive(children=(Dropdown(description='column1', options=('Energie_Betriebsst_BSZ_1_Kommentar_Betriebsstu…

# Plot des ganzen Jahres

Die folgenden Zellen verwenden den oben geladenen "Machine" spark dataframe. Pro Tag werden 3 Werte geschrieben:
- Der Maximalwert des Tages
- Der Minimalwert des Tages
- Der Durchschnittswert des Tages

Es kann im interaktiven Teil dann eine Spalte ausgewählt werden und diese drei Werte werden über den Kompletten Datenzeitraum geplotet

In [ ]:
exprs_max = {x: "max" for x in Machine.drop("timestamp").columns}
exprs_min = {x: "min" for x in Machine.drop("timestamp").columns}
exprs_avg = {x: "avg" for x in Machine.drop("timestamp").columns}

In [ ]:
Machine_max = Machine.withColumn("timestamp", F.date_trunc("day", "timestamp")).groupBy("timestamp").agg(exprs_max).toPandas()

In [ ]:
Machine_min = Machine.withColumn("timestamp", F.date_trunc("day", "timestamp")).groupBy("timestamp").agg(exprs_min).toPandas()

In [ ]:
Machine_avg = Machine.withColumn("timestamp", F.date_trunc("day", "timestamp")).groupBy("timestamp").agg(exprs_avg).toPandas()

In [ ]:
@interact 
def hallowelt_1(column = Machine.drop("timestamp").columns): 
    fig, ax1 = plt.subplots(figsize=(20,10))
    Machine_max.plot(x="timestamp", y= "max(" + column + ")", ax = ax1)
    Machine_min.plot(x="timestamp", y= "min(" + column + ")", ax = ax1)
    Machine_avg.plot(x="timestamp", y= "avg(" + column + ")", ax = ax1)

# Spark stoppen

Wenn Ihr das Notebook gerade nicht mehr braucht, bitte diese Zeile ausführen

In [10]:
spark.stop()